# &#x1F4DD; REPORT

In [1]:
## Receive the signal
include("../data/julia/rxsignal_withchannelandfreqoff.jl");

&#x1F4D1; Note: the `Julia` files translating the `MATLAB` files are located under the `../data/julia` directory. The `Julia` files are just reading the `MATLAB`'s files by using the `MAT` Julia Library. 

Below a Julia source code example opening the `.mat` MATLAB original file and reading the `rxs3` signal.

```julia
using MAT

ff = matopen("../data/matlab/rxsignal_withchannelandfreqoff.mat");

@read ff rxs3;
```

The original `MATLAB` data files are located under the `../data/matlab` folder and have `NOT` been modified nor altered. 

In [2]:
## Load the template signal
include("../data/julia/pss2.jl");

In [3]:
using FFTW
using DSP

In [4]:
## Load the useful math operations
include("../modules/operations.jl");

In [5]:
## Assign the received signal a variable Ŝᵣₓ³
𝑅ₚₛₛ³ = rxs3; # RX Received Signal 3 File Handle
@show size(𝑅ₚₛₛ³), typeof(𝑅ₚₛₛ³);

(size(𝑅ₚₛₛ³), typeof(𝑅ₚₛₛ³)) = ((616447, 1), Matrix{ComplexF64})


In [6]:
using LinearAlgebra

In [7]:
## Assign the template signal a variable 𝐻ₚₛₛ²
## convert the signal in time domain
Pₚₛₛ² = pss_2; # File Handle
# Hₚₛₛ²ᵀ = transpose(Pₚₛₛ²); # Hessian Transpose ?
𝐻ₚₛₛ² = Fᴵ(Pₚₛₛ²); # S Slanted (fourier transform) in time domain
𝐻ₚₛₛ² ./= norm(𝐻ₚₛₛ²);
𝐻ₚₛₛ² = 𝐻ₚₛₛ²[(end-143):end] ⧺ 𝐻ₚₛₛ²
@show size(𝐻ₚₛₛ²), typeof(𝐻ₚₛₛ²); # end is 2048 in this case, concat math 

(size(𝐻ₚₛₛ²), typeof(𝐻ₚₛₛ²)) = ((2192,), Vector{ComplexF64})


In [8]:
## Prepare the template signal for convolution
𝐻̅ₚₛₛ² = ⦰(𝐻ₚₛₛ²); # reverse #typed H\overbar
𝐻̅ₚₛₛ²ᴴ = conj(𝐻̅ₚₛₛ²); # conjugate 
@show length(𝐻̅ₚₛₛ²ᴴ), typeof(𝐻ₚₛₛ²);

(length(𝐻̅ₚₛₛ²ᴴ), typeof(𝐻ₚₛₛ²)) = (2192, Vector{ComplexF64})


In [10]:
## Perform the convolution between the 2 signals
𝐻̂ₚₛₛ² = 10 * log10.(abs.( 𝑅ₚₛₛ³ ⊗ 𝐻̅ₚₛₛ²ᴴ ))
@show size( 𝐻̂ₚₛₛ² ), typeof(𝐻̂ₚₛₛ²);  #typed \itH\hat

(size(𝐻̂ₚₛₛ²), typeof(𝐻̂ₚₛₛ²)) = ((618638, 1), Matrix{Float64})


In [11]:
# Find maximum value and its index
Ĉᵩ², 𝑁̂𝑓² = argmax(𝐻̂ₚₛₛ²)
@show Ĉᵩ², 𝑁̂𝑓²;

(Ĉᵩ², 𝑁̂𝑓²) = (50.649042476081405, CartesianIndex(6628, 1))


In [12]:
using Plots

In [13]:
# Plot the result
m2_chan_plot = 
plot(𝐻̂ₚₛₛ², xlabel="Sample", ylabel="Power (dB)", title="Convolution Result", ylim=(-20, 60))
savefig(m2_chan_plot,"images/m2_chan_plot.png");

<img src=images/m2_chan_plot.png width='' heigth='' > </img>

In [14]:
########################################################
# Frequency Offset Estimator Function
########################################################
function freq_offset_est(𝑅ₚₛₛ, 𝐻ₚₛₛ, Nf, m, 𝑓ₛ)

    # Frequency offset estimator
    Y = zeros(ComplexF64, length(m));
    L = length(𝐻ₚₛₛ)
    t = 0:(1/𝑓ₛ):((L-1)/𝑓ₛ)

    signal_part = 𝑅ₚₛₛ[Nf:(Nf + L -1)]; @show size( signal_part )
    for j = 1:length(m)
        Y[j] = Y[j] + abs.(∑(exp.(-2*pi*im*m[j].*t) .* conj(𝐻ₚₛₛ) .* ⦰(signal_part)))^2;
    end

    return Y
end

freq_offset_est (generic function with 1 method)

In [15]:
𝑓ₛ = 61.44e6 # Msamples/s.

Δ𝑓 = 10.0
m = -7500.:Δ𝑓:7500.

θ₁ = 144 # Cyclic Prefix
θ₂ = 2048 # FFT Size

2048

In [16]:
N𝑓 = getindex(𝑁̂𝑓², 1) - length(𝐻ₚₛₛ²) + 1
@show N𝑓, getindex(𝑁̂𝑓², 1) , length(𝐻ₚₛₛ²);

(N𝑓, getindex(𝑁̂𝑓², 1), length(𝐻ₚₛₛ²)) = (4437, 6628, 2192)


In [17]:
# Grab the function Profs Frequency Offset with his values
Y = freq_offset_est(𝑅ₚₛₛ³, 𝐻ₚₛₛ², N𝑓, m, 𝑓ₛ)
@show size(Y), typeof(Y);

size(signal_part) = (2192,)
(size(Y), typeof(Y)) = ((1501,), Vector{ComplexF64})


In [18]:
N̂𝑓, m̂ = argmax(abs.(Y))
@show m̂, 10 * log10(N̂𝑓);

(m̂, 10 * log10(N̂𝑓)) = (587, 88.46888484427753)


In [19]:
cfo_estim_plot = 
plot(10 * log10.(abs.(Y))
    , xlabel="Sample"
    , ylabel="Power (dB)"
    , title="Estimation Result"
);
# scatter!((m̂, (10 * log10(N̂𝑓))), color="red", label="m̂")
# savefig(cfo_estim_plot,"images/cfo_estim_plot.png");

<img src="images/cfo_estim_plot.png" width='' height='' > </img>

In [62]:
# Compensate CFO by multiplying the received signal with a complex exponential
function compensate_cfo(𝑅ₚₛₛ, N̂𝑓, 𝑓ₛ, Npss)
    # Generate a complex exponential with the specified frequency offset
    # 𝑅ₚₛₛ = 𝑅ₚₛₛ .* exp.(-2π * im * collect(1:4*Npss) * (N̂𝑓/𝑓ₛ))
    𝑅ₚₛₛ = 𝑅ₚₛₛ .* exp.(-2π * im * collect(1:4*Npss) * (N̂𝑓/𝑓ₛ))
    return 𝑅ₚₛₛ
end

compensate_cfo (generic function with 1 method)

In [63]:
headₛₛᵦ = N𝑓+θ₁; tailₛₛᵦ = headₛₛᵦ + (4 * (θ₁+θ₂)); @show headₛₛᵦ, tailₛₛᵦ;

(headₛₛᵦ, tailₛₛᵦ) = (4581, 13349)


In [64]:
𝑅̅ₚₛₛ = 𝑅ₚₛₛ³[headₛₛᵦ:tailₛₛᵦ-1]; @show size(𝑅̅ₚₛₛ);

size(𝑅̅ₚₛₛ) = (8768,)


In [65]:
# Tu = 2048 # Unit of samples
Tu = (θ₁+θ₂)

2192

In [66]:
Y = compensate_cfo(𝑅̅ₚₛₛ, N̂𝑓, 𝑓ₛ, Tu);

## PSS demodulation

In [67]:
using FFTW, DSP

In [68]:
# Start demodulation 
𝐹ₓ = DSP.fftshift(DSP.fft(Y)); @show size(𝐹ₓ), 𝐹ₓ[1];
# Correct for the fact that we're starting the demodulation at the middle of the cyclic prefix
# 𝐹ₓ .*= exp.(1im * 2π .* DSP.fftshift(DSP.fftfreq(Tu)) * Ncp / 2); 

(size(𝐹ₓ), 𝐹ₓ[1]) = ((8768,), 37804.69370138299 + 91491.57148331574im)


In [69]:
# The PSS ocuppies subcarriers 56 to 128 relative to the start of an SS/PBCH block
# In this case, the SS/PBCH block starts at subcarrier -120 (with respect to the
# subcarrier we have decided to treat as DC). This means that the SS/PBCH block is
# not centred with respect to this choice of DC (but this doesn't matter).
pss_sc_sel = range(1, 127)

1:127

In [70]:
x_all_sc = collect(0:size(𝐹ₓ,1)); y_all_sc = 10 * log10.(abs2.(𝐹ₓ));

In [71]:
x_pss_sc = collect(0:size(𝐹ₓ,1))[pss_sc_sel]; y_pss_sc = 10 * log10.(abs2.(𝐹ₓ[pss_sc_sel]));

In [74]:
all_sb_plot = 
scatter(x_all_sc, y_all_sc
    , label="All subcarriers"
    , markersize=5
)
scatter!(x_pss_sc, y_pss_sc
    , label="PSS subcarriers"
    , markershape=:vline
    , markersize=5
)

title!("PSS subcarrier power")
xlabel!("Subcarrier number (DC = 0)")
ylabel!("Power (dB)")

savefig(all_sb_plot,"images/all_sb_plot.png");

<img src="images/all_sb_plot.png" width='' height='' > </img>